## A/B Testing website conversion

A/B testing on a website to test whether the experimental yields a better conversion rate for a website than the current theme


The motivation is to understand and control the variables within the A/B test.

Note: this is a very simple experiment which does not factor in the funnel and journey to conversion or track bounce rate etc.

## 1. Designing the experiement

Formulating an experiment

Given we don’t know if the new design will perform better or worse (or the same?) as our current design, we’ll choose a two-tailed test:

Hₒ: p = pₒ

Hₐ: p ≠ pₒ

where p and pₒ stand for the conversion rate of the new and old design, respectively. We’ll also set a confidence level of 95%:


### Choosing the variables

For our test we’ll need two groups:

A control group - They'll be shown the old design

A treatment (or experimental) group - They'll be shown the new design


This will be our Independent Variable. 

The reason we have two groups even though we know the baseline conversion rate is that we want to control for other variables that could have an effect on our results: by having a control group we can directly compare their results to the treatment group, because the only systematic difference between the groups is the design of the landing page, and we can therefore attribute any differences in results to the designs.

For our Dependent Variable (i.e. what we are trying to measure), we are interested in capturing the conversion rate. A way we can code this is by each user session with a binary variable:

0 - The user did not buy the product during this user session

1 - The user bought the product during this user session

The number of people (or user sessions) we decide to capture in each group will have an effect on the precision of our estimated conversion rates: the larger the sample size, the more precise our estimates (i.e. the smaller our confidence intervals), the higher the chance to detect a difference in the two groups, if present.

### Choosing a sample size
It is important to note that since we won’t test the whole user base (our population), the conversion rates that we’ll get will inevitably be only estimates of the true rates.

The number of people (or user sessions) we decide to capture in each group will have an effect on the precision of our estimated conversion rates: the larger the sample size, the more precise our estimates (i.e. the smaller our confidence intervals), the higher the chance to detect a difference in the two groups, if present.

On the other hand, the larger our sample gets, the more expensive (and impractical) our study becomes.
So how many people should we have in each group?

In [ ]:
# Packages imports
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

%matplotlib inline

# Some plot styling preferences
plt.style.use('seaborn-whitegrid')
font = {'family' : 'Helvetica',
        'weight' : 'bold',
        'size'   : 14}

mpl.rc('font', **font)
effect_size = sms.proportion_effectsize(0.13, 0.15)    # Calculating effect size based on our expected rates

required_n = sms.NormalIndPower().solve_power(
    effect_size, 
    power=0.8, 
    alpha=0.05, 
    ratio=1
    )                                                  # Calculating sample size needed

required_n = ceil(required_n)                          # Rounding up to next whole number                          

print(required_n)

We’d need at least the number of observations for each group and have a 90% chance to test it is significantly significant

## 2. Collecting and preparing the data

In [ ]:
# importing the datas sample
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# sample of what the collated data looks like
df = pd.read_csv('/kaggle/input/ab-testing/ab_data.csv')

df.head()

In [ ]:
df.info

In [ ]:
# the dataset contains users that have experienced both the new and old pages.
pd.crosstab(df['group'], df['landing_page'])

In [ ]:
session_counts = df['user_id'].value_counts(ascending=False)
multi_users = session_counts[session_counts > 1].count()

print(f'There are {multi_users} users that appear multiple times in the dataset')

In [ ]:
# we drop the second time the instance used the homepage
users_to_drop = session_counts[session_counts > 1].index

df = df[~df['user_id'].isin(users_to_drop)]
print(f'The updated dataset now has {df.shape[0]} entries')

In [ ]:
# the dataset is now cleaner as can see the user experience only one page
pd.crosstab(df['group'], df['landing_page'])

### Sampling

We only sample the required amount the show from the power test.

We can use pandas' DataFrame.sample() method to do this, which will perform Simple Random Sampling for us.

In [ ]:
# here we collated the required n from each group as defined earlier

control_sample = df[df['group'] == 'control'].sample(n=required_n, random_state=22) 
treatment_sample = df[df['group'] == 'treatment'].sample(n=required_n, random_state=22)

ab_test = pd.concat([control_sample, treatment_sample], axis=0)
ab_test.reset_index(drop=True, inplace=True)
ab_test

In [ ]:
ab_test.info()

In [ ]:
ab_test['group'].value_counts()

## 3. Visualising the results

The first thing we can do is to calculate some basic statistics to get an idea of what our samples look like.

In [ ]:
conversion_rates = ab_test.groupby('group')['converted']

std_p = lambda x: np.std(x, ddof=0)              # Std. deviation of the proportion
se_p = lambda x: stats.sem(x, ddof=0)            # Std. error of the proportion (std / sqrt(n))

conversion_rates = conversion_rates.agg([np.mean, std_p, se_p])
conversion_rates.columns = ['conversion_rate', 'std_deviation', 'std_error']


conversion_rates.style.format('{:.3f}')

Judging by the stats above, it does look like our two designs performed very similarly, with our new design performing slightly better average conversion rate.

In [ ]:
plt.figure(figsize=(8,6))

sns.barplot(x=ab_test['group'], y=ab_test['converted'], ci=False)

plt.ylim(0, 0.17)
plt.title('Conversion rate by group')
plt.xlabel('Group')
plt.ylabel('Converted (proportion)');

## 4. Testing the hypothesis

In [ ]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
control_results = ab_test[ab_test['group'] == 'control']['converted']
treatment_results = ab_test[ab_test['group'] == 'treatment']['converted']
n_con = control_results.count()
n_treat = treatment_results.count()
successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treat]

z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'ci 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'ci 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')

## 5. Drawing Conclusion

Since our p-value is above our α=0.05 threshold, we cannot reject the Null hypothesis Hₒ, which means that our new design did not perform significantly different (let alone better) than our old one.

Additionally, if we look at the confidence interval for the treatment group we notice that:

It includes our baseline value of 13% conversion rate. It does not include our target value of 15% (the 2% uplift we were aiming for)

What this means is that it is more likely that the true conversion rate of the new design is similar to our baseline, rather than the 15% target we had hoped for. This is further proof that our new design is not likely to be an improvement on our old design, and that unfortunately we go back to redesign.

Guide from Renato of sample to 
https://medium.com/@RenatoFillinich/ab-testing-with-python-e5964dd66143
